# Requirements

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/pavelpopov/courses/digital image processing/project'

In [2]:
import math, re
import tensorflow as tf
import numpy as np
import glob

2023-11-20 21:24:43.738844: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Read tfrec

In [8]:
data_save_path = "./assets/data/np_data/data"
i = 0
for image, label in ds_train.take(1):
    data = image.numpy()
    labels = label.numpy()

    print(data.shape)
    print(labels.shape)
    np.savez(f"{data_save_path}_all.npz", data=data, labels=labels)
    i += 1

In [3]:
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

In [ ]:
print(glob.glob("/home/adam/*"))